In [58]:
import pandas as pd 
import numpy as np 
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [59]:
df= pd.read_excel("Merged_Students_Data_Final.xlsx")
df

,Student_ID,Name,Year,Email,Branch,Total_Classes,Classes_Attended,Internal_Test1_Score,Internal_Test2_Score,Internal_Test3_Score,Most_Failed_Subject,Fee_Status,Drop_Rate
0,S2001,Gabrielle Moore,Second,nicholas45@ramos.com,ME,60,37,12,5,1,DSA,UnPaid,3
1,S2002,Barbara Davies,Second,browngeorge@hotmail.com,ECE,60,27,11,12,4,OS,UnPaid,3
2,S2003,James Owens,Second,olarsen@serrano.biz,IT,60,31,5,8,12,OS,UnPaid,3
3,S2004,Barbara Simmons,Second,hernandezronald@wagner-smith.com,EE,60,50,10,9,20,DSA,Paid,2
4,S2005,Patrick Castro,Second,melissagarcia@hotmail.com,ECE,60,52,7,5,29,Networks,Paid,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,S2196,Charles Warren,Second,allenbrian@gmail.com,IT,60,58,29,22,26,Maths,Paid,1
196,S2197,Julie Giles,Second,bcarr@gmail.com,ECE,60,57,0,23,28,DSA,Paid,1
197,S2198,Marcus Page Jr.,Second,caseyjeanette@gmail.com,CIVIL,60,36,1,14,13,Networks,Paid,3
198,S2199,Molly Ramos,Second,jennifer71@kirby.com,ECE,60,48,16,4,14,DBMS,Paid,2


In [60]:
df["Attendance_Percentage"] = (df["Classes_Attended"] / df["Total_Classes"]) * 100
df.drop(["Name", "Year", "Student_ID", "Email", "Branch", "Classes_Attended", "Total_Classes","Most_Failed_Subject"], axis=1, inplace=True)
df["Average_Test_Score"] = df[["Internal_Test1_Score", "Internal_Test2_Score", "Internal_Test3_Score"]].mean(axis=1)


In [61]:
x_train, x_test, y_train, y_test= train_test_split(df.iloc[:, 3:], df["Drop_Rate"], random_state=42, train_size=0.8)
x_train.drop(["Drop_Rate"], axis=1, inplace=True)
x_test.drop(["Drop_Rate"], axis=1, inplace=True)
x_train, y_train

(    Fee_Status  Attendance_Percentage  Average_Test_Score
 79      UnPaid              71.666667           11.000000
 197       Paid              60.000000            9.333333
 38      UnPaid              60.000000           10.666667
 24        Paid              91.666667           13.000000
 122       Paid              98.333333           16.333333
 ..         ...                    ...                 ...
 106       Paid              96.666667           10.666667
 14        Paid              50.000000           17.333333
 92      UnPaid              95.000000           16.000000
 179       Paid              33.333333           25.666667
 102       Paid              61.666667           12.666667
 
 [160 rows x 3 columns],
 79     3
 197    3
 38     3
 24     2
 122    1
       ..
 106    2
 14     2
 92     2
 179    2
 102    3
 Name: Drop_Rate, Length: 160, dtype: int64)

In [62]:
oe= OrdinalEncoder(categories=[["UnPaid", "Paid"]])
x_train["Fee_Status"]= oe.fit_transform(x_train[["Fee_Status"]])
x_test["Fee_Status"]= oe.fit_transform(x_test[["Fee_Status"]])
x_train= x_train.astype(int)

In [63]:
x_test= x_test.astype(int)

In [64]:
x_test

,Fee_Status,Attendance_Percentage,Average_Test_Score
95,1,35,18
15,0,33,19
30,0,55,15
158,1,60,15
128,1,88,16
115,1,93,9
69,1,55,17
170,0,86,8
174,0,35,11
45,0,71,8


In [65]:
model = LogisticRegression(multi_class="multinomial", max_iter=1000)
model.fit(x_train, y_train)

c:\Users\pc\OneDrive\Desktop\hackathon\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,1000
,multi_class,'multinomial'


In [68]:
pred= model.predict(x_test)

accuracy_score(y_test, pred)

0.75

In [69]:
import pickle

# Save trained model to a file
with open("drop_rate_model.pkl", "wb") as f:
    pickle.dump(model, f)
